## Custom API Notebook

### Initialize the Node

In [1]:
import syft as sy
import pandas as pd
from typing import Any
from bq_credentials import bq_credentials
from bq_credentials import bq_credentials as public_mock_bq_credentials

node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Starting test-domain-1 server on 0.0.0.0:7879

SQLite Store Path:
!open file:///tmp/7bca415d13ed4ec881f0d0aede098dbb.sqlite

Waiting for server to start.Creating default worker image with tag='local-dev'
.Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=babcb2b35c3b473892a6ea623c5d2665 in_memory=True
Created default worker pool.
...Data Migrated to latest version !!!


INFO:     Started server process [32873]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7879 (Press CTRL+C to quit)


INFO:     127.0.0.1:39028 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:39040 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:39040 - "GET /api/v2/metadata HTTP/1.1" 200 OK
. Done.
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:39040 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:39040 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:39056 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

INFO:     127.0.0.1:39070 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39082 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39092 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39106 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:39106 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:39106 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:39106 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:39112 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:39116 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:42928 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:42932 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:42938 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:42938 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:42938 

### Criate a private custom API Endpoint by using the decorator

This allows node admin to create a new private endpoint by using only the decorator.

In [2]:
@sy.api_endpoint(
    path="first.query",
    settings=bq_credentials,
)
def private_code_endpoint(
    context,
    query: str,
) -> Any:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        query,
        project=context.settings["project_id"],
    )

    return rows.to_dataframe()


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=private_code_endpoint)
response

SyftSuccess: Endpoint successfully created.

In [3]:
domain_client.api.services.api.api_endpoints()

[syft.service.api.api.TwinAPIEndpointView]

### Create public/private Custom API Endpoint using TwinAPIEndpoint

This allows the admin to create a public/private endpoint interface where the users can iteract with.

In [4]:
@sy.public_api_endpoint(
    settings={"Hello": "Public"}
)
def public_function(
    context,
) -> str:
    return "Public Function Execution"

@sy.private_api_endpoint(
    settings={"Hello": "Private"}
)
def private_function(
    context,
) -> str:
    return "Private Function Execution"

new_endpoint = sy.TwinAPIEndpoint(
    path="third.query",
    private=private_function,
    public=public_function,
    description="Lore ipsulum ...",
)

# # Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_endpoint)
response

SyftSuccess: Endpoint successfully created.

### Using new Custom Endpoints

In [5]:
domain_client = node.login(email="info@openmined.org", password="changethis")

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

### Private ones

In [6]:
domain_client.api.services.first.query

## API: first.query
#### Description: <span style="font-weight: normal;">Lorem ipsum dolor sit amet lorem adipiscing elit …</span><br>
##### Private Code:
```python
N / A
```
##### Public Code:
```python
def private_code_endpoint(
    context,
    query: str,
) -> Any:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        query,
        project=context.settings["project_id"],
    )

    return rows.to_dataframe()

```


In [7]:
domain_client.api.services.first.query(query="SELECT *  FROM test_1gb.subreddits LIMIT 100")

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False


In [8]:
domain_client.register(
    email="user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

Logged into <test-domain-1: High-side Domain> as GUEST
Logged into <test-domain-1: High side Domain> as <user@openmined.org>


In [11]:
domain_guest.api.services.first.query(query="SELECT *  FROM test_1gb.subreddits LIMIT 100")

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False


### Public/Private ones

In [12]:
domain_client.api.services.third.query

## API: third.query
#### Description: <span style="font-weight: normal;">Lorem ipsum dolor sit amet lorem adipiscing elit …</span><br>
##### Private Code:
```python
N / A
```
##### Public Code:
```python
def public_function(
    context,
) -> str:
    return "Public Function Execution"

```


In [19]:
domain_client.api.services.third.query()

'Private Function Execution'

In [16]:
domain_guest.api.services.third.query()

'Public Function Execution'

In [20]:
domain_client.api.services.api.delete(endpoint_path="first.query")

SyftError: Exception calling api.delete. Traceback (most recent call last):
  File "/home/ionesio/workspace/PySyft/packages/syft/src/syft/node/node.py", line 1267, in handle_api_call_with_unsigned_result
    result = method(context, *api_call.args, **api_call.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ionesio/workspace/PySyft/packages/syft/src/syft/service/service.py", line 365, in _decorator
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ionesio/workspace/PySyft/packages/syft/src/syft/service/api/api_service.py", line 121, in delete
    result = self.stash.delete_by_uid(context.credentials, endpoint.id)
                                                           ^^^^^^^^^^^
AttributeError: 'Ok' object has no attribute 'id'

In [21]:
domain_client.api.services.api.api_endpoints()

[syft.service.api.api.TwinAPIEndpointView,
 syft.service.api.api.TwinAPIEndpointView]

In [ ]:
# Local Execution
domain_guest.api.services.third.query.private()


# Remote Function
@sy.single_use(domain):
def foo(domain):
    domain.api.services.services.third.query()



client.code.submit(foo)
client.code.foo()